# YOLOv8-ES Training on Kaggle

Complete notebook for training YOLOv8-ES on Kaggle with free GPU

## Cell 1: Install and Setup

In [ ]:
import os
import sys
from pathlib import Path
import yaml

print("=" * 70)
print("YOLOv8-ES Training on Kaggle")
print("=" * 70)
print()

print("📦 Installing dependencies...")
!pip install -q ultralytics

# Add yolov8es modules to path if they exist in output
if Path('/kaggle/working/yolov8es').exists():
    sys.path.insert(0, '/kaggle/working')
    print("✅ YOLOv8-ES modules found in output directory")
else:
    print("⚠️  YOLOv8-ES modules not found (training baseline only)")

print("✅ Setup complete!")
print()

## Cell 2: Verify Installation

In [ ]:
print("🔍 Verifying modules...")

import torch
from ultralytics import YOLO

print("✅ PyTorch loaded")
print("✅ Ultralytics loaded")

# Try to load YOLOv8-ES modules if available
try:
    from yolov8es.model.edcm import EDCM
    from yolov8es.model.sgam import SGAM
    from yolov8es.model.loss_wiou import WIoUv3Loss
    print("✅ EDCM module loaded")
    print("✅ SGAM module loaded")
    print("✅ WIoU v3 module loaded")
except ImportError:
    print("⚠️  YOLOv8-ES modules not found (will train baseline YOLOv8n)")

print()
print("All modules ready!")
print()

## Cell 3: Configure Dataset

In [ ]:
print("⚙️  Configuring dataset...")

# ⚠️ UPDATE THESE PATHS TO YOUR KAGGLE DATASET
DATASET_PATH = '/kaggle/input/crackathon-data/randomized_dataset'  # <-- CHANGE THIS!
TRAIN_IMAGES = 'images/train'  # Relative path to training images
VAL_IMAGES = 'images/val'      # Relative path to validation images

# Check if dataset exists
if not Path(DATASET_PATH).exists():
    print(f"⚠️  Dataset not found at: {DATASET_PATH}")
    print("Available datasets:")
    !ls /kaggle/input/
    print("\nPlease update DATASET_PATH variable above")
else:
    print(f"✅ Dataset found: {DATASET_PATH}")
    print("\nDataset structure:")
    !ls -la {DATASET_PATH}
    
    # Check for images directory
    if Path(f"{DATASET_PATH}/images").exists():
        print("\nImages directory:")
        !ls -la {DATASET_PATH}/images/
    else:
        print("\n⚠️  No 'images' directory found. Checking structure...")
        !find {DATASET_PATH} -type d -name "*train*" -o -name "*val*" | head -10

# Create dataset configuration
dataset_config = {
    'path': DATASET_PATH,
    'train': TRAIN_IMAGES,
    'val': VAL_IMAGES,
    'nc': 5,  # Number of classes (0-4)
    'names': {
        0: 'D00',  # Longitudinal crack
        1: 'D10',  # Transverse crack
        2: 'D20',  # Alligator crack
        3: 'D43',  # Other corruption
        4: 'D44'   # Pothole
    }
}

# Save configuration
config_path = '/kaggle/working/dataset.yaml'
with open(config_path, 'w') as f:
    yaml.dump(dataset_config, f)

print(f"\n✅ Dataset config saved: {config_path}")
print("\nConfig contents:")
!cat {config_path}
print()

## Cell 4: Check GPU

In [ ]:
print("🖥️  Checking GPU availability...")

import torch

if torch.cuda.is_available():
    print(f"✅ CUDA available")
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    device = 0
else:
    print("⚠️  No GPU detected!")
    print("   Enable GPU: Settings → Accelerator → GPU T4 x2")
    device = 'cpu'

print()

## Cell 5: Train YOLOv8-ES with Custom Modules

In [ ]:
print("=" * 70)
print("🚀 Training YOLOv8-ES with EDCM + SGAM + WIoU")
print("=" * 70)
print()

from ultralytics import YOLO
from ultralytics.models.yolo.detect import DetectionTrainer
from yolov8es.model.edcm import EDCM
from yolov8es.model.sgam import SGAM
from yolov8es.model.loss_wiou import WIoUv3Loss
import torch
import torch.nn as nn

# Create custom trainer with WIoU v3 loss and module integration
class YOLOv8ESTrainer(DetectionTrainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.wiou_loss = WIoUv3Loss()
    
    def get_model(self, cfg=None, weights=None, verbose=True):
        """Get model with custom EDCM and SGAM modules."""
        # Get base model
        model = super().get_model(cfg, weights, verbose)
        
        # Get device from args
        device = self.device
        
        # Replace backbone layer 2 with EDCM
        backbone_layer = model.model[2]
        c1 = backbone_layer.cv1.conv.in_channels
        c2 = backbone_layer.cv2.conv.out_channels
        model.model[2] = EDCM(c1, c2).to(device)
        print(f"✅ Replaced backbone layer 2 with EDCM ({c1} -> {c2})")
        
        # Add SGAM to neck after layer 12
        neck_layer = model.model[12]
        c_neck = neck_layer.cv2.conv.out_channels
        sgam = SGAM(c_neck).to(device)
        model.model[12] = nn.Sequential(neck_layer, sgam)
        print(f"✅ Added SGAM to neck after layer 12 (channels: {c_neck})")
        
        return model
    
    def criterion(self, preds, batch):
        """Compute loss with WIoU v3 for bounding boxes."""
        # Get standard loss components
        loss = super().criterion(preds, batch)
        
        # Replace box loss with WIoU v3
        if hasattr(loss, 'box'):
            # Extract predictions and targets
            pred_bboxes = preds[0] if isinstance(preds, (list, tuple)) else preds
            
            # Get target bboxes from batch
            if 'bboxes' in batch:
                target_bboxes = batch['bboxes']
                
                # Compute WIoU v3 loss
                try:
                    wiou_loss_value = self.wiou_loss(pred_bboxes, target_bboxes)
                    # Replace box loss component
                    loss.box = wiou_loss_value * self.args.box
                except Exception as e:
                    print(f"⚠️  WIoU calculation failed, using standard IoU: {e}")
        
        return loss

# Load base YOLOv8n model
print("Loading YOLOv8n base model...")
model = YOLO('yolov8n.pt')

print("✅ Base model loaded")
print("✅ Custom trainer configured with EDCM + SGAM + WIoU v3!")
print()

# Training parameters
train_args = {
    'data': config_path,
    'epochs': 100,
    'batch': 16,
    'imgsz': 640,
    'device': device,
    'project': '/kaggle/working/runs',
    'name': 'yolov8es',
    'exist_ok': True,
    'pretrained': True,
    'optimizer': 'SGD',
    'lr0': 0.01,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'box': 7.5,
    'cls': 0.5,
    'dfl': 1.5,
    'plots': True,
    'save': True,
    'save_period': 10,
    'val': True,
    'cache': False,
    'workers': 8,
    'amp': False,
}

print("Training configuration:")
for key, value in train_args.items():
    print(f"  {key}: {value}")
print()

print("🎯 Training YOLOv8-ES with all three improvements:")
print("   1. EDCM (Enhanced Dynamic Convolution Module)")
print("   2. SGAM (Selective Global Attention Mechanism)")
print("   3. WIoU v3 (Wise-IoU v3 Loss)")
print()

# Start training with custom trainer
print("Starting training...")
print("Modules will be integrated when trainer initializes model...")
print()

results = model.train(trainer=YOLOv8ESTrainer, **train_args)

print()
print("=" * 70)
print("✅ Training Complete!")
print("=" * 70)
print(f"Results saved to: {results.save_dir}")
print()

## Cell 6: Validate Model

In [ ]:
print("📊 Validating model...")

metrics = model.val()

print()
print("=" * 70)
print("Validation Results")
print("=" * 70)
print(f"mAP50:     {metrics.box.map50:.4f}")
print(f"mAP50-95:  {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall:    {metrics.box.mr:.4f}")
print("=" * 70)
print()

## Cell 7: Save Results

In [ ]:
print("💾 Saving results...")

from shutil import make_archive, copy2

# Copy best weights to easy location
best_weights = '/kaggle/working/runs/yolov8es/weights/best.pt'
last_weights = '/kaggle/working/runs/yolov8es/weights/last.pt'

if Path(best_weights).exists():
    copy2(best_weights, '/kaggle/working/best.pt')
    print("✅ Best weights: /kaggle/working/best.pt")

if Path(last_weights).exists():
    copy2(last_weights, '/kaggle/working/last.pt')
    print("✅ Last weights: /kaggle/working/last.pt")

# Create zip of all results
print("Creating results archive...")
make_archive('/kaggle/working/training_results', 'zip', '/kaggle/working/runs')
print("✅ Results archive: /kaggle/working/training_results.zip")

print()
print("=" * 70)
print("📥 Download Files")
print("=" * 70)
print("Go to Output tab (right sidebar) and download:")
print("  • best.pt - Best model weights")
print("  • last.pt - Last checkpoint")
print("  • training_results.zip - All training results")
print("=" * 70)
print()

## Cell 8: Test Inference (Optional)

In [ ]:
print("🔍 Testing inference on validation images...")

# Load best model
model = YOLO('/kaggle/working/best.pt')

# Predict on a few validation images
val_images = f'{DATASET_PATH}/{VAL_IMAGES}'

if Path(val_images).exists():
    results = model.predict(
        source=val_images,
        save=True,
        conf=0.25,
        iou=0.7,
        max_det=300,
        project='/kaggle/working/predictions',
        name='test',
        exist_ok=True,
    )
    
    print(f"✅ Predictions saved to: /kaggle/working/predictions/test")
    print(f"   Processed {len(results)} images")
else:
    print(f"⚠️  Validation images not found at: {val_images}")

print()
print("=" * 70)
print("🎉 All Done!")
print("=" * 70)
print()
print("Next steps:")
print("1. Download your trained model (best.pt)")
print("2. Download results (training_results.zip)")
print("3. Check predictions in /kaggle/working/predictions/test")
print()
print("To train YOLOv8-ES with EDCM and SGAM, see:")
print("  docs/KAGGLE_TRAINING_GUIDE.md")